In [ ]:
import pandas as pd
import numpy as np
from mip import *
from pulp import *
import csv
from pulp import GLPK
import matplotlib.pyplot as plt
import math
import statistics
from pytictoc import TicToc
import time
from ttictoc import tic,toc

In [ ]:
# Supplier profile
Supplier_profile = pd.read_csv("monthly suppliers profile.csv")
display(Supplier_profile)

In [ ]:
# Suppliers profile

B = 17 # set of biomass
S = 77 # set of suppliers
T = 12 # set of time-step
Supplier_Profile=np.zeros([B,S,T])
for s in range(S):
    for b in range(B):
        for t in range(T):
                Supplier_Profile[b,s,t] = Supplier_profile.iloc[(17*s)+b,t+2]

In [ ]:
# Power plant profile
PowerPlant_profile = pd.read_csv("plants profile.csv")
PowerPlant_profile

In [ ]:
# Distance
Distance = pd.read_csv("distance.csv")
display(Distance)

In [ ]:
# Biomass data
Biomass_data = pd.read_csv("biomass.csv")
display(Biomass_data)

In [ ]:
num_days_in_month = np.array([31,28,31,30,31,30,31,31,30,31,30,31])

# ==========================================================
# Optimize with the lowest maximum inventory level
# ==========================================================

In [ ]:
tic()


C_TF = 455542.86 # Fixed transportation cost per a truck
D_Eth = 4790000 #L/day
D_BElc = 2550.405 # MW (target at 3940 MW, eal capacity = 3000.477)
D_GElc = 35.65874 # MW (target at 387 MW, real capacity = 403.504)
C_IF = 0 #THB/ton

# PuLP
# Set==========================================================================
B = 17 # set of biomass
S = 77 # set of suppliers
PQ123 = 215 # set of plant
PQ4 = 185
PQ5 = 27
T = 12 # set of time period
# D = 77 #set of demand


prob = LpProblem(name = 'BioenergySC', sense = LpMinimize)

# Variables ==============================================================

Xb123 = LpVariable.dicts("Amount_of_biomass_delivered_to_plant_withQ123",((b,s,p,t) for b in range(B) for s in range(S) for p in range (PQ123) for t in range(T)),lowBound=0, cat='Continuous')
Xb4 = LpVariable.dicts("Amount_of_biomass_delivered_to_plant_withQ4",((b,s,p,t) for b in range(B) for s in range(S) for p in range (PQ4) for t in range(T)),lowBound=0, cat='Continuous')
Xb5 = LpVariable.dicts("Amount_of_biomass_delivered_to_plant_withQ5",((b,s,p,t) for b in range(B) for s in range(S) for p in range (PQ5) for t in range(T)),lowBound=0, cat='Continuous')

Xp123=LpVariable.dicts("Amount_of_biomass_fed_into_process_withQ123",((b,p,t) for b in range(B) for p in range (PQ123) for t in range(T)),lowBound=0, cat='Continuous')
Xp4=LpVariable.dicts("Amount_of_biomass_fed_into_process_withQ4",((b,p,t) for b in range(B) for p in range (PQ4) for t in range(T)),lowBound=0, cat='Continuous')
Xp5=LpVariable.dicts("Amount_of_biomass_fed_into_process_withQ5",((b,p,t) for b in range(B) for p in range (PQ5) for t in range(T)),lowBound=0, cat='Continuous')

I123 = LpVariable.dicts("Inventory_withQ123",((b,p,t) for b in range(B) for p in range (PQ123) for t in range(T)),lowBound=0, cat='Continuous')
I4 = LpVariable.dicts("Inventory_withQ4",((b,p,t) for b in range(B) for p in range (PQ4) for t in range(T)),lowBound=0, cat='Continuous')
I5 = LpVariable.dicts("Inventory_withQ5",((b,p,t) for b in range(B) for p in range (PQ5) for t in range(T)),lowBound=0, cat='Continuous')
Max_I123 = LpVariable.dicts("Maximum Inventory Level_withQ123", ((p) for p in range (PQ123)), lowBound=0, cat='Continuous')
Max_I4 = LpVariable.dicts("Maximum Inventory Level_withQ4", ((p) for p in range (PQ4)), lowBound=0, cat='Continuous')
Max_I5 = LpVariable.dicts("Maximum Inventory Level_withQ5", ((p) for p in range (PQ5)), lowBound=0, cat='Continuous')

# Objective ==========================================================================

prob += lpSum(Max_I123[p] for p in range (PQ123))+lpSum(Max_I4[p] for p in range (PQ4))+lpSum(Max_I5[p] for p in range (PQ5))
    
# Constraints==========================================================================

### Limitation of biomass
for b in range(B):
    for s in range(S):
        for t in range(T):
            prob+= lpSum(Xb123[b,s,p,t] for p in range(PQ123))+lpSum(Xb4[b,s,p,t] for p in range(PQ4))+lpSum(Xb5[b,s,p,t] for p in range(PQ5)) <= Supplier_Profile[b,s,t]
            
### Limitation of inventory
for p in range(PQ123):
    for t in range(T):
        prob+=lpSum(I123[b,p,t] for b in range(B))<= Max_I123[p]
for p in range(PQ4):
    for t in range(T):
        prob+=lpSum(I4[b,p,t] for b in range(B))<= Max_I4[p]
for p in range(PQ5):
    for t in range(T):
        prob+=lpSum(I5[b,p,t] for b in range(B))<= Max_I5[p]

    
###  Inventory change
for b in range(B):
    for p in range(PQ123):
        for t in range(T):
            if t==0:
                prob += I123[b,p,t] == lpSum(Xb123[b,s,p,t] for s in range(S)) - Xp123[b,p,t]
            else:
                prob+= I123[b,p,t] == I123[b,p,t-1]+lpSum(Xb123[b,s,p,t] for s in range(S)) - Xp123[b,p,t]
                
for b in range(B):
    for p in range(PQ4):
        for t in range(T):
            if t==0:
                prob += I4[b,p,t] == lpSum(Xb4[b,s,p,t] for s in range(S)) - Xp4[b,p,t]
            else:
                prob+= I4[b,p,t] == I4[b,p,t-1]+lpSum(Xb4[b,s,p,t] for s in range(S)) - Xp4[b,p,t]
                
for b in range(B):
    for p in range(PQ5):
        for t in range(T):
            if t==0:
                prob += I5[b,p,t] == lpSum(Xb5[b,s,p,t] for s in range(S)) - Xp5[b,p,t]
            else:
                prob+= I5[b,p,t] == I5[b,p,t-1]+lpSum(Xb5[b,s,p,t] for s in range(S)) - Xp5[b,p,t]
                
### Biomass fed into process cannot exceed inventory current level
for b in range(B):
    for p in range(PQ123):
        for t in range(T):
            if t==0:
                prob+= Xp123[b,p,t] <= lpSum(Xb123[b,s,p,t] for s in range(S)) 
            else:
                prob+= Xp123[b,p,t] <= I123[b,p,t-1]+ lpSum(Xb123[b,s,p,t] for s in range(S)) 
                
for b in range(B):
    for p in range(PQ4):
        for t in range(T):
            if t==0:
                prob+= Xp4[b,p,t] <= lpSum(Xb4[b,s,p,t] for s in range(S)) 
            else:
                prob+= Xp4[b,p,t] <= I4[b,p,t-1]+ lpSum(Xb4[b,s,p,t] for s in range(S))
                
for b in range(B):
    for p in range(PQ5):
        for t in range(T):
            if t==0:
                prob+= Xp5[b,p,t] <= lpSum(Xb5[b,s,p,t] for s in range(S)) 
            else:
                prob+= Xp5[b,p,t] <= I5[b,p,t-1]+ lpSum(Xb5[b,s,p,t] for s in range(S))
                
## Limitation of electricity and bioethanol produced
for p in range(PQ123):
    for t in range(T):
        prob+= PowerPlant_profile['Electricity coef (MWh/MJ)'][p]*lpSum(Xp123[b,p,t]*Biomass_data['Heat Capacity (MJ/ton)'][b] for b in range(B)) <= PowerPlant_profile['Maximum electricity production capacity (MW)'][p]*24*num_days_in_month[t]*PowerPlant_profile['maximum plant factor'][p]

for p in range(PQ4):
    for t in range(T):
        prob+= PowerPlant_profile['Electricity coef (MWh/MJ)'][p+PQ123]*lpSum(Xp4[b,p,t]*Biomass_data['Methane heat eq (MJ/ton)'][b] for b in range(B)) <= PowerPlant_profile['Maximum electricity production capacity (MW)'][p+PQ123]*24*num_days_in_month[t]*PowerPlant_profile['maximum plant factor'][p]
        
for p in range(PQ5):
    for t in range(T):
        prob+= lpSum(Xp5[b,p,t]*Biomass_data['Bioethanol coef (L/ton)'][b] for b in range(B)) <= PowerPlant_profile['Maximum bioethanol production capacity (L/day)'][p+PQ123+PQ4]*num_days_in_month[t]*PowerPlant_profile['maximum plant factor'][p+PQ123+PQ4]

### Biomass to Electricity demand constraint
for t in range(T):
    prob+= lpSum(PowerPlant_profile['Electricity coef (MWh/MJ)'][p]*lpSum(Xp123[b,p,t]*Biomass_data['Heat Capacity (MJ/ton)'][b] for b in range(B)) for p in range(PQ123)) == D_BElc*24*num_days_in_month[t]/365

### Biogas to Electricity demand constraint
for t in range(T):
    prob+= lpSum(PowerPlant_profile['Electricity coef (MWh/MJ)'][p+PQ123]*lpSum(Xp4[b,p,t]*Biomass_data['Methane heat eq (MJ/ton)'][b] for b in range(B)) for p in range(PQ4)) == D_GElc*24*num_days_in_month[t]/365
### Ethanol demand constraint
for t in range(T):
    prob+= lpSum(Xp5[b,p,t]*Biomass_data['Bioethanol coef (L/ton)'][b] for b in range(B) for p in range(PQ5)) == D_Eth*num_days_in_month[t]

                                                                                
status=prob.solve() 

elapsed = toc()
print('Elapsed time:',elapsed)

In [ ]:
# Optimal value ====================================================================
print("Optimization status: ",LpStatus[status])
print("Optimal Objective value: ", value(prob.objective), '  tons')
print('\n Total cost :    ', lpSum(Biomass_data['Biomass cost (THB/ton)'][b]*Xb123[b,s,p,t].varValue for b in range(B) for s in range (S) for p in range(PQ123) for t in range(T))+lpSum(Biomass_data['Biomass cost (THB/ton)'][b]*Xb4[b,s,p,t].varValue for b in range(B) for s in range (S) for p in range(PQ4) for t in range(T))+lpSum(Biomass_data['Biomass cost (THB/ton)'][b]*Xb5[b,s,p,t].varValue for b in range(B) for s in range (S) for p in range(PQ5) for t in range(T))+lpSum(Biomass_data['Transportation cost (THB/ton.km)'][b]*Xb123[b,s,p,t].varValue*Distance.iloc[p,s+1] for b in range(B) for s in range (S) for p in range (PQ123) for t in range (T))+lpSum(Biomass_data['Transportation cost (THB/ton.km)'][b]*Xb4[b,s,p,t].varValue*Distance.iloc[p+PQ123,s+1] for b in range(B) for s in range (S) for p in range (PQ4) for t in range (T))+lpSum(Biomass_data['Transportation cost (THB/ton.km)'][b]*Xb5[b,s,p,t].varValue*Distance.iloc[p+PQ123+PQ4,s+1] for b in range(B) for s in range (S) for p in range (PQ5) for t in range (T))+lpSum(PowerPlant_profile['Electricity production cost (LCOE)(THB/MWh)'][p]*PowerPlant_profile['Electricity coef (MWh/MJ)'][p]*lpSum(Xp123[b,p,t].varValue*Biomass_data['Heat Capacity (MJ/ton)'][b] for b in range(B)) for p in range(PQ123) for t in range (T))+lpSum(PowerPlant_profile['Electricity production cost (LCOE)(THB/MWh)'][p+PQ123]*PowerPlant_profile['Electricity coef (MWh/MJ)'][p+PQ123]*lpSum(Xp4[b,p,t].varValue*Biomass_data['Methane heat eq (MJ/ton)'][b] for b in range(B)) for p in range(PQ4) for t in range (T))+lpSum(PowerPlant_profile['Bioethanol production cost (THB/L)'][p+PQ123+PQ4]*lpSum(Xp5[b,p,t].varValue*Biomass_data['Bioethanol coef (L/ton)'][b] for b in range(B)) for p in range(PQ5) for t in range (T))+lpSum(lpSum(I123[b,p,t].varValue*0.1*Biomass_data['Biomass cost (THB/ton)'][b] for b in range(B)) for p in range (PQ123) for t in range (T))+lpSum(lpSum(I4[b,p,t].varValue*0.1*Biomass_data['Biomass cost (THB/ton)'][b] for b in range(B)) for p in range (PQ4) for t in range (T))+lpSum(lpSum(I5[b,p,t].varValue*0.1*Biomass_data['Biomass cost (THB/ton)'][b] for b in range(B)) for p in range (PQ5) for t in range (T))+lpSum(C_IF*Max_I123[p].varValue for p in range(PQ123))+lpSum(C_IF*Max_I4[p].varValue for p in range(PQ4))+lpSum(C_IF*Max_I5[p].varValue for p in range(PQ5)), '   THB')
print("\t Total biomass material cost  =  ",  lpSum(Biomass_data['Biomass cost (THB/ton)'][b]*Xb123[b,s,p,t].varValue for b in range(B) for s in range (S) for p in range (PQ123) for t in range(T))+lpSum(Biomass_data['Biomass cost (THB/ton)'][b]*Xb4[b,s,p,t].varValue for b in range(B) for s in range (S) for p in range (PQ4) for t in range(T))+lpSum(Biomass_data['Biomass cost (THB/ton)'][b]*Xb5[b,s,p,t].varValue for b in range(B) for s in range (S) for p in range (PQ5) for t in range(T)))
print("\t Total transportation  cost  =  " , lpSum(Biomass_data['Transportation cost (THB/ton.km)'][b]*Xb123[b,s,p,t].varValue*Distance.iloc[p,s+1] for b in range(B) for s in range (S) for p in range (PQ123) for t in range (T))+lpSum(Biomass_data['Transportation cost (THB/ton.km)'][b]*Xb4[b,s,p,t].varValue*Distance.iloc[p+PQ123,s+1] for b in range(B) for s in range (S) for p in range (PQ4) for t in range (T))+lpSum(Biomass_data['Transportation cost (THB/ton.km)'][b]*Xb5[b,s,p,t].varValue*Distance.iloc[p+PQ123+PQ4,s+1] for b in range(B) for s in range (S) for p in range (PQ5) for t in range (T)))
print("\t Total holding cost  =  " ,lpSum(lpSum(I123[b,p,t].varValue*0.1*Biomass_data['Biomass cost (THB/ton)'][b] for b in range(B)) for p in range (PQ123) for t in range (T))+lpSum(lpSum(I4[b,p,t].varValue*0.1*Biomass_data['Biomass cost (THB/ton)'][b] for b in range(B)) for p in range (PQ4) for t in range (T))+lpSum(lpSum(I5[b,p,t].varValue*0.1*Biomass_data['Biomass cost (THB/ton)'][b] for b in range(B)) for p in range (PQ5) for t in range (T)))
print("\t Total electricity from biomass production cost  =  " , lpSum(PowerPlant_profile['Electricity production cost (LCOE)(THB/MWh)'][p]*PowerPlant_profile['Electricity coef (MWh/MJ)'][p]*lpSum(Xp123[b,p,t].varValue*Biomass_data['Heat Capacity (MJ/ton)'][b] for b in range(B)) for p in range(PQ123) for t in range (T)))
print("\t Total electricity from biogas production cost  =  " , lpSum(PowerPlant_profile['Electricity production cost (LCOE)(THB/MWh)'][p+PQ123]*PowerPlant_profile['Electricity coef (MWh/MJ)'][p+PQ123]*lpSum(Xp4[b,p,t].varValue*Biomass_data['Methane heat eq (MJ/ton)'][b] for b in range(B)) for p in range(PQ4) for t in range (T)))     
print("\t Total bioethanol production cost  =  " , lpSum(PowerPlant_profile['Bioethanol production cost (THB/L)'][p+PQ123+PQ4]*lpSum(Xp5[b,p,t].varValue*Biomass_data['Bioethanol coef (L/ton)'][b] for b in range(B)) for p in range(PQ5) for t in range (T)))


In [ ]:
# Optimal variables

Optimal_Xb = np.zeros((B,S,PQ123+PQ4+PQ5,T))
for b in range(B):
    for s in range (S):
        for p in  range(PQ123+PQ4+PQ5):
            for t in range(T):
                if p < PQ123:
                    Optimal_Xb[b,s,p,t] = Xb123[b,s,p,t].varValue
                elif PQ123 <=p < (PQ123+PQ4):
                    Optimal_Xb[b,s,p,t] = Xb4[b,s,p-PQ123,t].varValue
                elif (PQ123+PQ4) <=p < (PQ123+PQ4+PQ5):
                    Optimal_Xb[b,s,p,t] = Xb5[b,s,p-PQ123-PQ4,t].varValue

Optimal_Xb_sumP = np.zeros((B,S,T))
for b in range(B):
    for s in range (S):
        for t in range(T):
            Optimal_Xb_sumP[b,s,t] = np.sum(Optimal_Xb[b,s,:,t],axis=0)

                    
Optimal_Xb_sumS = np.zeros((B,PQ123+PQ4+PQ5,T))
for b in range(B):
    for t in range(T):
        for p in  range(PQ123+PQ4+PQ5):
            Optimal_Xb_sumS[b,p,t] = np.sum(Optimal_Xb[b,:,p,t],axis=0)
            
Optimal_Xb_sumBS = np.zeros((PQ123+PQ4+PQ5,T))
for p in range(PQ123+PQ4+PQ5):
    for t in range(T):
        Optimal_Xb_sumBS[p,t] = np.sum(Optimal_Xb_sumS[:,p,t],axis=0)
                
Optimal_Xb_sumSP = np.zeros((B,T))
for b in range(B):
    for t in range(T):
        Optimal_Xb_sumSP[b,t] = np.sum(Optimal_Xb_sumP[b,:,t],axis=0)

Optimal_Xb_sumBSP = np.zeros(T)
for t in range(T):
    Optimal_Xb_sumBSP[t] = np.sum(Optimal_Xb_sumBS[:,t],axis=0)
            
Optimal_Xp = np.zeros((B,PQ123+PQ4+PQ5,T))
for b in range(B):
    for p in  range(PQ123+PQ4+PQ5):
        for t in range(T):
            if p < PQ123:
                Optimal_Xp[b,p,t] = Xp123[b,p,t].varValue
            elif PQ123 <=p < (PQ123+PQ4):
                Optimal_Xp[b,p,t] = Xp4[b,p-PQ123,t].varValue
            elif  (PQ123+PQ4) <=p <  (PQ123+PQ4+PQ5):
                Optimal_Xp[b,p,t] = Xp5[b,p-PQ123-PQ4,t].varValue
                
Optimal_Xp_sumB = np.zeros((PQ123+PQ4+PQ5,T))
for p in  range(PQ123+PQ4+PQ5):
    for t in range(T):
        Optimal_Xp_sumB[p,t] = np.sum(Optimal_Xp[:,p,t],axis=0)

Optimal_Xp_sumBP = np.zeros(T)
for t in range(T):
    Optimal_Xp_sumBP[t] = np.sum(Optimal_Xp_sumB[:,t],axis=0)        
            
Optimal_I= np.zeros((B,PQ123+PQ4+PQ5,T))
for b in range(B):
    for p in  range(PQ123+PQ4+PQ5):
        for t in range(T):
            if p < PQ123:
                Optimal_I[b,p,t] = I123[b,p,t].varValue
            elif PQ123 <=p < (PQ123+ PQ4):
                Optimal_I[b,p,t] = I4[b,p-PQ123,t].varValue
            elif  (PQ123+ PQ4)<=p <  (PQ123+ PQ4+PQ5):
                Optimal_I[b,p,t] = I5[b,p-PQ123-PQ4,t].varValue
            
Optimal_I_sumB = np.zeros((PQ123+PQ4+PQ5,T))
for p in  range(PQ123+PQ4+PQ5):
    for t in range(T):
        Optimal_I_sumB[p,t] = np.sum(Optimal_I[:,p,t],axis=0)
        
Optimal_I_sumBP = np.zeros(T)
for t in range(T):
    Optimal_I_sumBP[t] = np.sum(Optimal_I_sumB[:,t],axis=0)
    
Optimal_I_sumBT = np.zeros(PQ123+PQ4+PQ5)
for p in range(PQ123+PQ4+PQ5):
    Optimal_I_sumBT[p] = np.sum(Optimal_I_sumB[p,:],axis=0)
        
Optimal_Max_I = np.zeros((PQ123+PQ4+PQ5,T))
for p in range(PQ123+PQ4+PQ5):
    for t in range(T):
        if p < PQ123:
            Optimal_Max_I[p,t]=Max_I123[p].varValue
        elif PQ123 <=p < PQ123+PQ4:
            Optimal_Max_I[p,t]=Max_I4[p-PQ123].varValue
        elif PQ4<=p < PQ123+PQ4+PQ5:
            Optimal_Max_I[p,t]=Max_I5[p-PQ123-PQ4].varValue
            
Optimal_Max_I_sumP = np.zeros(T)
for t in range(T):
    Optimal_Max_I_sumP[t]=np.sum(Optimal_Max_I[:,t],axis=0)
            
Optimal_EnergyProduce = np.zeros((B,PQ123+PQ4+PQ5,T))
for p in range(PQ123+PQ4+PQ5):  
    for t in range(T):
        for b in range(B):
            if p < PQ123:
                Optimal_EnergyProduce[b,p,t] = PowerPlant_profile['Electricity coef (MWh/MJ)'][p]*Optimal_Xp[b,p,t]*Biomass_data['Heat Capacity (MJ/ton)'][b]
            elif PQ123 <=p < (PQ123+ PQ4):
                Optimal_EnergyProduce[b,p,t] = PowerPlant_profile['Electricity coef (MWh/MJ)'][p]*Optimal_Xp[b,p,t]*Biomass_data['Methane heat eq (MJ/ton)'][b]
            elif  (PQ123+ PQ4)<=p <  (PQ123+ PQ4+PQ5):
                Optimal_EnergyProduce[b,p,t] = Optimal_Xp[b,p,t]*Biomass_data['Bioethanol coef (L/ton)'][b]
                
                
Optimal_EnergyProduce_sumB = np.zeros((PQ123+PQ4+PQ5,T))
for p in range(PQ123+PQ4+PQ5):  
    for t in range(T):
        Optimal_EnergyProduce_sumB[p,t] = np.sum(Optimal_EnergyProduce[:,p,t],axis=0)
        
Optimal_BElcProduce_sumBP = np.zeros(T)
for t in range(T):
    Optimal_BElcProduce_sumBP[t] = np.sum(Optimal_EnergyProduce_sumB[0:PQ123,t],axis=0)

Optimal_GElcProduce_sumBP = np.zeros(T)
for t in range(T):
    Optimal_GElcProduce_sumBP[t] = np.sum(Optimal_EnergyProduce_sumB[PQ123:PQ123+PQ4,t],axis=0)

Optimal_EthProduce_sumBP = np.zeros(T)
for t in range(T):
    Optimal_EthProduce_sumBP[t] = np.sum(Optimal_EnergyProduce_sumB[PQ123+PQ4:PQ123+PQ4+PQ5,t],axis=0)

In [ ]:
# Total usage of biomass type b
timestep = np.linspace(1.0, 12.0, num=12)
Max_B=np.zeros((B,T))


for b in range(B):
    for t in range(T):
#         for s in range(S):
        Max_B[b,t] = np.sum(Supplier_Profile[b,:,t],axis=0)
#             Max_B[b,t] += Supplier_Profile[b,s,t]
        
for b in range(B):
    plt.plot(timestep,Optimal_Xb_sumSP[b,:],label='Total usage of biomass type {}'.format(b+1))
    plt.plot(timestep,Max_B[b,:],'--',label='Maximum available of biomass type {}'.format(b+1))
    plt.legend(loc="best",bbox_to_anchor=(0.8, -0.2))
    plt.title('Biomass type {} usage '.format(b+1))
    plt.xlabel('Timestep (month)')
    plt.ylabel('Amount (tons)')
            #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
    plt.show()


In [ ]:
# Total usage of biomass type b
timestep = np.linspace(1.0, 12.0, num=12)
        
for b in range(B):
    plt.plot(timestep,Optimal_Xb_sumSP[b,:],label='Total usage of biomass type {}'.format(b+1))
    plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
    plt.title('Biomass usage ')
    plt.xlabel('Timestep (month)')
    plt.ylabel('Amount (tons)')
            #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()


In [ ]:
# Total usage of all biomass 
timestep = np.linspace(1.0, 12.0, num=12)
Max_B_sumB=np.zeros(T)


for t in range(T):
    Max_B_sumB[t] = np.sum(np.sum(Supplier_Profile[:,:,t],axis=0),axis=0)

        
plt.plot(timestep,Optimal_Xb_sumBSP[:],label='Total usage of all biomass type ')
plt.plot(timestep,Max_B_sumB[:],'--',label='Maximum available of all biomass type ')
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Total biomass usage '.format(b+1))
plt.xlabel('Timestep (month)')
plt.ylabel('Amount (tons)')
#plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()


In [ ]:
# Biomass fed into energy production process

timestep = np.linspace(1.0, 12.0, num=12)

for p in range(PQ123+PQ4+PQ5):
    
    plt.plot(timestep,Optimal_Xp_sumB[p,:],label='Total biomass fed into process at plant {}'.format(p+1))
    plt.plot(timestep,Optimal_Xb_sumBS[p,:],'*',label='Total biomass delivered at plant {}'.format(p+1))
    plt.plot(timestep,Optimal_I_sumB[p,:],'--',label='Inventory level of plant {}'.format(p+1))
    plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
    plt.title('Total biomass fed into process at of plant {}'.format(p+1))
    plt.xlabel('Timestep (month)')
    plt.ylabel('Amount (tons)')
            #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
    plt.show()

In [ ]:
# Total Biomass fed into energy production process

timestep = np.linspace(1.0, 12.0, num=12)

plt.plot(timestep,Optimal_Xb_sumBSP[:],label='Total usage of all biomass type (tons)')    
plt.plot(timestep,Optimal_Xp_sumBP[:],label='Total biomass fed into process at plant (tons)')
plt.plot(timestep,((D_Eth*0.0059313)+((D_BElc+D_GElc)*24)*num_days_in_month[:]),label='Total energy demand (MWh)')
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Total biomass fed into process')
plt.xlabel('Timestep (month)')

            #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()

In [ ]:
# Inventory level and its maximum capacity

timestep = np.linspace(1.0, 12.0, num=12)

Variance_I = np.zeros(PQ123+PQ4+PQ5)
for p in range(PQ123+PQ4+PQ5):
    Variance_I[p] = statistics.pvariance(Optimal_I_sumB[p,:])


for p in range(PQ123+PQ4+PQ5):
    print('Populational Variance of inventory of plant {}  :'.format(p+1), Variance_I[p])
    plt.plot(timestep,Optimal_I_sumB[p,:],label='Inventory level of plant {}'.format(p+1))
    plt.plot(timestep,Optimal_Max_I[p,:],label='Maximum inventory level of plant {}'.format(p+1))
    plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
    plt.title('Inventory level of plant {}'.format(p+1))
    plt.xlabel('Timestep (month)')
    plt.ylabel('Amount (tons)')
            #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
    plt.show()

In [ ]:

plt.plot(timestep,np.sum(Optimal_I_sumB[0:PQ123,:],axis=0),label='Inventory level of biomass power plant')
plt.plot(timestep,np.sum(Optimal_Max_I[0:PQ123,:],axis=0),label='Maximum inventory level of biomass power plants ')
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Inventory level of biomass power plants')
plt.xlabel('Timestep (month)')
plt.ylabel('Amount (tons)')
            #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()

plt.plot(timestep,np.sum(Optimal_I_sumB[PQ123:PQ123+PQ4,:],axis=0),label='Inventory level of biogas power plants')
plt.plot(timestep,np.sum(Optimal_Max_I[PQ123:PQ123+PQ4,:],axis=0),label='Maximum inventory level of biogas power plants ')
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Inventory level of biogas power plants')
plt.xlabel('Timestep (month)')
plt.ylabel('Amount (tons)')
            #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()

plt.plot(timestep,np.sum(Optimal_I_sumB[PQ123+PQ4:PQ123+PQ4+PQ5,:],axis=0),label='Inventory level of bioethanol plants')
plt.plot(timestep,np.sum(Optimal_Max_I[PQ123+PQ4:PQ123+PQ4+PQ5,:],axis=0),label='Maximum inventory level of bioethanol plants ')
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Inventory level of bioethanol plants')
plt.xlabel('Timestep (month)')
plt.ylabel('Amount (tons)')
            #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()

In [ ]:
# Inventory level and its maximum capacity

timestep = np.linspace(1.0, 12.0, num=12)

Variance_I_sumP = statistics.pvariance(Optimal_I_sumBP[:])
Variance_I_sumPQ123 = statistics.pvariance(Optimal_I_sumBT[:PQ123])
Variance_I_sumPQ4 = statistics.pvariance(Optimal_I_sumBT[PQ123:PQ123+PQ4])
Variance_I_sumPQ5 = statistics.pvariance(Optimal_I_sumBT[PQ123+PQ4:PQ123+PQ4+PQ5])

print('Populational Variance of inventory of all plant    :', Variance_I_sumP)
print('Populational Variance of inventory of biomass power plants    :', Variance_I_sumPQ123)
print('Populational Variance of inventory of biogas power plants    :', Variance_I_sumPQ4)
print('Populational Variance of inventory of bioethanol plants    :', Variance_I_sumPQ5)
print('Inventory utilization of all plant    :', np.sum(np.sum(Optimal_I_sumB[0:PQ123+PQ4+PQ5,:],axis=0),axis=0)/np.sum(np.sum(Optimal_Max_I[0:PQ123+PQ4+PQ5,:],axis=0),axis = 0))
print('Inventory utilization of biomass power plants    :', np.sum(np.sum(Optimal_I_sumB[0:PQ123,:],axis=0),axis=0)/np.sum(np.sum(Optimal_Max_I[0:PQ123,:],axis=0),axis = 0))
print('Inventory utilization of biogas power plants    :', np.sum(np.sum(Optimal_I_sumB[PQ123:PQ123+PQ4,:],axis=0),axis=0)/np.sum(np.sum(Optimal_Max_I[PQ123:PQ123+PQ4,:],axis=0),axis = 0))
print('Inventory utilization of bioethanol plants    :', np.sum(np.sum(Optimal_I_sumB[PQ123+PQ4:PQ123+PQ4+PQ5,:],axis=0),axis=0)/np.sum(np.sum(Optimal_Max_I[PQ123+PQ4:PQ123+PQ4+PQ5,:],axis=0),axis = 0))
plt.plot(timestep,np.sum(Optimal_I_sumB[0:PQ123+PQ4+PQ5,:],axis=0),label='Total inventory level')
plt.plot(timestep,np.sum(Optimal_Max_I[0:PQ123+PQ4+PQ5,:],axis=0),label='Total maximum inventory level')
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Total inventory level')
plt.xlabel('Timestep (month)')
plt.ylabel('Amount (tons)')
            #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()

In [ ]:
# Energy produced at plant p 
timestep = np.linspace(1.0, 12.0, num=12)
Max_E=np.zeros((PQ123+PQ4+PQ5,T))


for t in range(T):
    for p in range(PQ123+PQ4+PQ5):
        if p < PQ123+PQ4:
            Max_E[p,t] += PowerPlant_profile['Maximum electricity production capacity (MW)'][p]*3600*24*0.85*num_days_in_month[t]
        elif PQ123+PQ4<=p < PQ123+PQ4+PQ5:
            Max_E[p,t] += PowerPlant_profile['Maximum bioethanol production capacity (L/day)'][p]*num_days_in_month[t]*0.85
            
            
for p in range(PQ123+PQ4+PQ5):
    print('Capacity factor:   ', np.sum(Optimal_EnergyProduce_sumB[p,:],axis=0)/np.sum(Max_E[p,:],axis=0))
    plt.plot(timestep,Optimal_EnergyProduce_sumB[p,:],label='Energy produced at plant {}'.format(p+1))
    plt.plot(timestep,Max_E[p,:],label='Maximum energy production cap at plant {}'.format(p+1))
    plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
    plt.title('Energy produced at plant {}'.format(p+1))
    plt.xlabel('Timestep (month)')
    if p < PQ123+PQ4:
        plt.ylabel('Energy (MWh)')
    elif PQ123+PQ4<=p < PQ123+PQ4+PQ5:
        plt.ylabel('Energy (Liters)')
    #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
    plt.show()

In [ ]:
timestep = np.linspace(1.0, 12.0, num=12)
D_BElc_sumP_monthly = np.zeros(T)
D_GElc_sumP_monthly = np.zeros(T)
D_Eth_sumP_monthly = np.zeros(T)
for t in range(T):
    D_BElc_sumP_monthly[t] = D_BElc*num_days_in_month[t]*24
    D_GElc_sumP_monthly[t] = D_GElc*num_days_in_month[t]*24
    D_Eth_sumP_monthly[t] = D_Eth*num_days_in_month[t]
    
Max_BElcProdCap= np.zeros(T)
Max_GElcProdCap = np.zeros(T)
Max_EthProdCap = np.zeros(T)
for t in range(T):
    for p in range(PQ123+PQ4+PQ5):
        if p < PQ123:
            Max_BElcProdCap[t] += PowerPlant_profile['Maximum electricity production capacity (MW)'][p]*24*num_days_in_month[t]*PowerPlant_profile['maximum plant factor'][p]
        if PQ123 <= p < PQ123+PQ4:
            Max_GElcProdCap[t] += PowerPlant_profile['Maximum electricity production capacity (MW)'][p]*24*num_days_in_month[t]*PowerPlant_profile['maximum plant factor'][p]
        if PQ123+PQ4 <= p < PQ123+PQ4+PQ5:
            Max_EthProdCap[t] += PowerPlant_profile['Maximum bioethanol production capacity (L/day)'][p]*num_days_in_month[t]*PowerPlant_profile['maximum plant factor'][p]

print('Capacity factor of all biomass power plants:   ', np.sum(Optimal_BElcProduce_sumBP[:],axis=0)/np.sum(Max_BElcProdCap[:],axis=0))    
print('Capacity factor of all biogas power plants:   ', np.sum(Optimal_GElcProduce_sumBP[:],axis=0)/np.sum(Max_GElcProdCap[:],axis=0))    
print('Capacity factor of all bioethanol plants:   ', np.sum(Optimal_EthProduce_sumBP[:],axis=0)/np.sum(Max_EthProdCap[:],axis=0))
            
plt.plot(timestep,Optimal_BElcProduce_sumBP[:],'--',label='Energy produced of all biomass power plants')
plt.plot(timestep,Max_BElcProdCap[:],':',label='Maximum energy production capacity of all biomass power plants')
plt.plot(timestep, D_BElc_sumP_monthly[:],label='Electricity demand from biomass')
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Energy produced of all biomass power plants')
plt.xlabel('Timestep (month)')
plt.ylabel('Energy (MWh)')
# plt.ylabel('Energy (Liters)')
    #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()

plt.plot(timestep,Optimal_GElcProduce_sumBP[:],'--',label='Energy produced of all biogas power plants')
plt.plot(timestep,Max_GElcProdCap[:],':',label='Maximum energy production capacity of all biogas power plants')
plt.plot(timestep, D_GElc_sumP_monthly[:],label='Electricity demand from biogas')     
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Energy produced of all biogas power plants')
plt.xlabel('Timestep (month)')
plt.ylabel('Energy (MWh)')
# plt.ylabel('Energy (Liters)')
    #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()

plt.plot(timestep,Optimal_EthProduce_sumBP[:],'--',label='Energy produced of all bioethanol plants')
plt.plot(timestep,Max_EthProdCap[:],':',label='Maximum energy production capacity of all bioethanol plants')
plt.plot(timestep, D_Eth_sumP_monthly[:],label='Bioethanol demand')     
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Energy produced of all bioethanol plants')
plt.xlabel('Timestep (month)')
# plt.ylabel('Energy (MWh)')
plt.ylabel('Energy (Liters)')
    #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()            



In [ ]:
print('Capacity factor of all biomass power plants:   ', np.sum(Optimal_BElcProduce_sumBP[:]+Optimal_GElcProduce_sumBP[:]+(Optimal_EthProduce_sumBP[:]*0.0059313),axis=0)/np.sum(Max_BElcProdCap[:]+Max_GElcProdCap[:]+(Max_EthProdCap[:]*0.0059313),axis=0))    
plt.plot(timestep,Optimal_BElcProduce_sumBP[:]+Optimal_GElcProduce_sumBP[:]+(Optimal_EthProduce_sumBP[:]*0.0059313),'--',label='Energy produced from all plants')
plt.plot(timestep,Max_BElcProdCap[:]+Max_GElcProdCap[:]+(Max_EthProdCap[:]*0.0059313),':',label='Maximum energy production capacity of all plants')
plt.plot(timestep, D_BElc_sumP_monthly[:]+D_GElc_sumP_monthly[:]+(D_Eth_sumP_monthly[:]*0.0059313),label='Energy demand')
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Energy produced of all biomass power plants')
plt.xlabel('Timestep (month)')
plt.ylabel('Energy (MWh)')
# plt.ylabel('Energy (Liters)')
    #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()

In [ ]:
# Energy produced of all biomass power plants'

timestep = np.linspace(1.0, 12.0, num=12)
D_BElc_sumP_monthly = np.zeros(T)
D_GElc_sumP_monthly = np.zeros(T)
D_Eth_sumP_monthly = np.zeros(T)
for t in range(T):
    D_BElc_sumP_monthly[t] = D_BElc*num_days_in_month[t]*24
    D_GElc_sumP_monthly[t] = D_GElc*num_days_in_month[t]*24
    D_Eth_sumP_monthly[t] = D_Eth*num_days_in_month[t]
    
plt.plot(timestep,Optimal_BElcProduce_sumBP[:],'--',label='Energy produced of all biomass power plants')
plt.plot(timestep, D_BElc_sumP_monthly[:],label='Electricity from biomass demand')
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Energy produced of all biomass power plants')
plt.xlabel('Timestep (day)')
plt.ylabel('Energy (MWh)')
# plt.ylabel('Energy (Liters)')
    #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()

plt.plot(timestep,Optimal_GElcProduce_sumBP[:],'--',label='Energy produced of all biogas power plants')
plt.plot(timestep, D_GElc_sumP_monthly[:],label='Electricity from biogas demand')     
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Energy produced of all biogas power plants')
plt.xlabel('Timestep (day)')
plt.ylabel('Energy (MWh)')
# plt.ylabel('Energy (Liters)')
    #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()

plt.plot(timestep,Optimal_EthProduce_sumBP[:],'--',label='Energy produced of all bioethanol plants')
plt.plot(timestep, D_Eth_sumP_monthly[:],label='Bioethanol demand')     
plt.legend(loc="best",bbox_to_anchor=(1.05, 1))
plt.title('Energy produced of all bioethanol plants')
plt.xlabel('Timestep (month)')
# plt.ylabel('Energy (MWh)')
plt.ylabel('Energy (Liters)')
    #plt.savefig('Amount of each biomass type, delivered from each supplier to each power plant.png',dpi=300)
plt.show()            

